In [ ]:
texto = """
A Microsoft foi fundada em 4 de abril de 1975 por Bill Gates e Paul Allen, em Albuquerque, 
no estado do Novo México. O nome "Microsoft" é uma combinação das palavras "microcomputador" 
e "software", refletindo o foco da empresa em software para computadores pessoais.
O primeiro grande projeto da Microsoft foi a criação de um sistema operacional para o 
computador Altair 8800, um dos primeiros microcomputadores disponíveis comercialmente. 
O sistema, denominado Altair BASIC, foi desenvolvido em parceria com a MITS 
(Micro Instrumentation and Telemetry Systems) e foi um marco inicial para a Microsoft.
Em 1980, a empresa firmou um contrato significativo com a IBM para fornecer o sistema 
operacional para o novo PC da IBM, o que levou à criação do MS-DOS. Esse contrato foi um 
ponto de virada para a Microsoft, impulsionando sua expansão e dominando o mercado de 
sistemas operacionais para PCs nos anos seguintes.
Com o sucesso do MS-DOS, a Microsoft se consolidou como líder no setor de software e, 
em 1985, lançou o Windows, um sistema operacional gráfico que viria a se tornar a base 
de sua supremacia no mercado de sistemas operacionais para desktop.
"""

In [ ]:
from langchain.pydantic_v1 import BaseModel, Field
from typing import List
from langchain_core.utils.function_calling import convert_to_openai_function

In [ ]:
class Event(BaseModel):
    """Informações sobre um evento ocorrido"""
    date: str = Field(description="Data do evento no formato YYYY-MM-DD")
    event: str = Field(description="Descrição do evento extraído do texto")
    
class EventsList(BaseModel):
    """Lista de Eventos para Extração"""
    events: List[Event] = Field(description="Conjunto de eventos encontrados no texto fornecido")
    
tool_events = convert_to_openai_function(EventsList)
tool_events

In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_messages([
    ("system", "Extraia as frases de acontecimentos e as extraia integralmente"),
    ("user", "{input}")
])

chat = ChatOpenAI()

chain = (prompt | chat.bind(functions=[tool_events],
                            function_call={"name":"EventsList"}))

In [ ]:
chain.invoke({"input": texto})

In [ ]:
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser

chain = (prompt 
         | chat.bind(functions=[tool_events], function_call={'name': 'EventsList'})
         | JsonOutputFunctionsParser())

chain.invoke({'input': texto})

### Dados da Web

In [ ]:
from langchain_community.document_loaders.web_base import WebBaseLoader

loader = WebBaseLoader("https://www.techtudo.com.br")
page = loader.load()
page

In [ ]:
class BlogPost(BaseModel):
    """Detalhes sobre uma postagem de blog"""
    title: str = Field(description="Título da postagem no blog")
    author: str = Field(description="Nome do autor da postagem no blog")
    
class BlogSite(BaseModel):
    """Conjunto de postagens de blog de um site específico"""
    posts: List[BlogPost] = Field(description="Coleção de postagens de blog do site")
    
tool_blog = convert_to_openai_function(BlogSite)
tool_blog

In [ ]:
from langchain.output_parsers.openai_functions import JsonKeyOutputFunctionsParser
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_messages([
    ('system', 'Extraia da página os posts do blog com as informações especificadas'),
    ('user', '{input}')
])
chat = ChatOpenAI()
chain = (prompt 
         | chat.bind(functions=[tool_blog], function_call={'name': 'BlogSite'})
         | JsonKeyOutputFunctionsParser(key_name='posts'))

In [ ]:
chain.invoke({"input": page})